In [1]:
import os
import pandas as pd
import time
from datetime import date, timedelta, datetime
from google.cloud import bigquery
# para extrair informações de páginas HTML
from bs4 import BeautifulSoup
# para paginas dinamicas
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import zipfile
import pytz
from flask import Flask
from unidecode import unidecode

In [11]:
chrome_options = webdriver.ChromeOptions()
# Criando um driver
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)

C:\Users\ricar\AppData\Local\Temp\ipykernel_25900\135824865.py:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


In [6]:
# Definindo as variáveis
PROJECT = 'bigdata-dados-publicos'
dataset = "DS_PORTAL_TRANSPARENCIA"

In [23]:
def get_zip():
    try:
        print("cheguei getzip")
        all_files = filter(os.path.isfile, os.listdir( os.curdir ))
        files_zip = list(filter(lambda f: f.endswith('.zip'), all_files))
        print(f"files_zip: {files_zip}")
        for i in files_zip:
            with zipfile.ZipFile(i, 'r') as zip_ref:
                zip_ref.extractall()
        print("terminei")
    except Exception as e:
        print(f"Something went wrong with scrapping, files could not be downloaded")
        print(e)
        raise e

In [23]:
def tratamento(csv_file):
    df = pd.read_csv(csv_file, encoding='latin-1', sep = ';')

    # Tirando acentos nos nomes das colunas e espaços
    for i in list(df.columns):
        new_column = unidecode(i).replace(" ", "_")
        if ")" in new_column:
            new_column = new_column.replace(")", '_')
        if "(" in new_column: 
            new_column = new_column.replace("(", '_')
        if "ç" in new_column:
            new_column = new_column.replace("ç", 'c')
        if "$" in new_column:
            new_column = new_column.replace("$", 'S')
        if "/" in new_column:
            new_column = new_column.replace("/", '_')
        if "-" in new_column:
            new_column = new_column.replace("-", '')
        if "?" in new_column:
            new_column = new_column.replace("?", '')
        df = df.rename(columns={f'{i}': f'{new_column.replace("__","_").replace("_ ", "")}'})

    df.to_csv(csv_file, index=False)

    print("Tratamento incluido", csv_file)

In [2]:
def job_config():
    config = bigquery.LoadJobConfig(schema = [
        bigquery.SchemaField("CODIGO_FAVORECIDO", "STRING"),
        bigquery.SchemaField("NOME_FAVORECIDO", "STRING"),
        bigquery.SchemaField("SIGLA_UF", "STRING"),
        bigquery.SchemaField("NOME_MUNICIPIO", "STRING"),
        bigquery.SchemaField("CODIGO_ORGAO_SUPERIOR", "STRING"),
        bigquery.SchemaField("NOME_ORGAO_SUPERIOR", "STRING"),
        bigquery.SchemaField("CODIGO_ORGAO", "STRING"),
        bigquery.SchemaField("NOME_ORGAO", "STRING"),
        bigquery.SchemaField("CODIGO_UNIDADE_GESTORA", "STRING"),
        bigquery.SchemaField("NOME_UNIDADE_GESTORA", "STRING"),
        bigquery.SchemaField("ANO_E_MES_DO_LANCAMENTO", "STRING"),
        bigquery.SchemaField("VALOR_RECEBIDO", "STRING")
    ])
    config.source_format = bigquery.SourceFormat.CSV
    config.autodetect = False
    config.create_disposition = "CREATE_IF_NEEDED"
    config.field_delimiter = ","
    config.skip_leading_rows = 1
    config.write_disposition = "WRITE_APPEND"
    config.allow_jagged_rows = True
    config.allow_quoted_newlines = True
    config.max_bad_records = 100000
    config.ignore_unknown_values = True
    return config

In [3]:
def bigquery_load(list_of_infos):
    client = bigquery.Client(project=PROJECT)
    print(os.getcwd())
    print(os.listdir( os.curdir ))
    all_files = filter(os.path.isfile, os.listdir( os.curdir ))
    files = list(filter(lambda f: f.endswith('.csv'), all_files))
    print(f"CSVs found{files}")
    
    for file_infos in list_of_infos:
        file_name = file_infos
        table_name = 'TB_DESPESAS_FAVORECIDOS'
        dataset = 'DS_PORTAL_TRANSPARENCIA'
        print(f"Starting {file_name} with the following infos {file_infos}...")
        try:
            file = str(list(filter(lambda f: f.startswith(file_name), files))[0])
            print(f"Found the correspoding file {file}")
            with open(file, "rb") as source_file:
                print(f"Loading {file} into BigQuery {dataset}.{table_name}")
                table = PROJECT + "." + dataset + "." + table_name
                load_job = client.load_table_from_file(source_file, table, job_config=job_config())
                load_job.result()
        except Exception as e:
            print(f"Something went wrong with dict {file_infos}, {file_name} couldn't be loaded into {dataset}.{table_name}")
            print(e)
            raise e
        

In [4]:
def call_functions():
    all_files = filter(os.path.isfile, os.listdir( os.curdir ))
    files_csv = list(filter(lambda f: f.endswith('.csv'), all_files))
    bigquery_load(files_csv)
    return "Job complete"

In [7]:
call_functions()

C:\Users\ricar\OneDrive - Insper - Institudo de Ensino e Pesquisa\Setimo_Semestre\BIGDATA\PROJETO
['.ipynb_checkpoints', '201912_RecebimentosRecursosPorFavorecido.csv', '202001_RecebimentosRecursosPorFavorecido.csv', '202002_RecebimentosRecursosPorFavorecido.csv', '202003_RecebimentosRecursosPorFavorecido.csv', '202004_RecebimentosRecursosPorFavorecido.csv', '202005_RecebimentosRecursosPorFavorecido.csv', '202006_RecebimentosRecursosPorFavorecido.csv', '202007_RecebimentosRecursosPorFavorecido.csv', '202008_RecebimentosRecursosPorFavorecido.csv', '202009_RecebimentosRecursosPorFavorecido.csv', '202010_RecebimentosRecursosPorFavorecido.csv', '202011_RecebimentosRecursosPorFavorecido.csv', '202012_RecebimentosRecursosPorFavorecido.csv', '202101_RecebimentosRecursosPorFavorecido.csv', '202102_RecebimentosRecursosPorFavorecido.csv', '202103_RecebimentosRecursosPorFavorecido.csv', '202104_RecebimentosRecursosPorFavorecido.csv', '202105_RecebimentosRecursosPorFavorecido.csv', '202106_Recebim

Starting 202103_RecebimentosRecursosPorFavorecido.csv with the following infos 202103_RecebimentosRecursosPorFavorecido.csv...
Found the correspoding file 202103_RecebimentosRecursosPorFavorecido.csv
Loading 202103_RecebimentosRecursosPorFavorecido.csv into BigQuery DS_PORTAL_TRANSPARENCIA.TB_DESPESAS_FAVORECIDOS
Starting 202104_RecebimentosRecursosPorFavorecido.csv with the following infos 202104_RecebimentosRecursosPorFavorecido.csv...
Found the correspoding file 202104_RecebimentosRecursosPorFavorecido.csv
Loading 202104_RecebimentosRecursosPorFavorecido.csv into BigQuery DS_PORTAL_TRANSPARENCIA.TB_DESPESAS_FAVORECIDOS
Starting 202105_RecebimentosRecursosPorFavorecido.csv with the following infos 202105_RecebimentosRecursosPorFavorecido.csv...
Found the correspoding file 202105_RecebimentosRecursosPorFavorecido.csv
Loading 202105_RecebimentosRecursosPorFavorecido.csv into BigQuery DS_PORTAL_TRANSPARENCIA.TB_DESPESAS_FAVORECIDOS
Starting 202106_RecebimentosRecursosPorFavorecido.csv w

'Job complete'